# SCohenLab Volumetric Image Processing notebook (Simplified MCZ)

--------------
# PIPELINE OVERVIEW
## 1. GOAL SETTING

### GOAL:  Infer sub-cellular components in order to understand interactome 

To measure shape, position, size, and interaction of eight organelles/cellular components (Nuclei (NU), Lysosomes (LS),Mitochondria (MT), Golgi (GL), Peroxisomes (PO), Endoplasmic Reticulum (ER), Lipid Droplet (LD), and SOMA) during differentiation of iPSCs, in order to understand the Interactome / Spatiotemporal coordination.

### summary of _OBJECTIVES_
- Infer subcellular objects:
  -  #### #1. [infer NUCLEI ](#image-processing-objective-1-infer-nucleii) - NU, ch 0
  -  #### #2. [Infer SOMA](#image-processing-objective-2-infer-soma) (extent of single brightest cell)- SO, composite icluding ch 1, ch 4,ch 5, and ch 7 "residuals"
  -  #### #3. [Infer CYTOSOL](#image-processing-objective-3-infer-cytosol)- CY 
  -  #### #4. [Infer LYSOSOMES](#image-processing-objective-4-infer-lysosomes)  - LS, ch 1
  -  #### #5. [Infer MITOCHONDRIA](#image-processing-objective-5-infer-mitochondria) - MT, ch 2
  -  #### #6. [Infer GOLGI complex](#image-processing-objective-6-infer-golgi-complex) - GL, ch 3
  -  #### #7. [Infer PEROXISOMES](#image-processing-objective-7-infer-peroxisomes) - PO, ch  4
  -  #### #8. [Infer ENDOPLASMIC RETICULUM ](#image-processing-objective-8-infer-endoplasmic-reticulum)- ER, ch 5
  -  #### #9. [Infer LB](#image-processing-objective-9-infer-lipid-bodies-droplet), LD, ch 6 





## 2. DATA CREATION
> METHODS
> iPSC lines prepared and visualized on Zeiss Microscopes. 32 channel multispectral images collected.  Linear Unmixing in  ZEN Blue software with target emission spectra yields 8 channel image outputs.  Channels correspond to: Nuclei (NU), Lysosomes (LS),Mitochondria (MT), Golgi (GL), Peroxisomes (PO), Endoplasmic Reticulum (ER), Lipid Droplet (LD), and a “residual” signal.

> Meta-DATA
>   - Microcope settings
>  - OME scheme
> - Experimenter observations
> - Sample, bio-replicate, image numbers, condition values, etc
>  - Dates
>  - File structure, naming conventions
>  - etc.





## 3. IMAGE PROCESSING
### INFERENCE OF SUB-CELLULAR OBJECTS
The imported images have already been pre-processed to transform the 32 channel spectral measuremnts into "linearly unmixed" images which estimate independently labeled sub-cellular components.  Thes 7 channels (plus a residual "non-linear" signal) will be used to infer the shapes and extents of these sub-cellular components.   
We will perform computational image analysis on the pictures (in 2D an 3D) to _segment_ the components of interest for measurement.  In other prcoedures we can used these labels as "ground truth" labels to train machine learning models to automatically perform the inference of these objects.
Pseudo-independent processing of the imported multi-channel image to acheive each of the 9 objecives stated above.  i.e. infering: NUCLEI, SOMA, CYTOSOL, LYSOSOME, MITOCHONDRIA, GOLGI COMPLEX, PEROZISOMES, ENDOPLASMIC RETICULUM, and LIPID BODIES

### General flow for infering objects via segmentation
- Pre-processing
- Core-processing (thresholding)
- Post-processing 

### QC




## 4. QUANTIFICATION

SUBCELLULAR COMPONENT METRICS
-  extent 
-  size
-  shape
-  position



### NOTE: PIPELINE TOOL AND DESIGN CHOICES?
We want to leverage the Allen Cell & Structure Setmenter.  It has been wrapped as a [napari-plugin](https://www.napari-hub.org/plugins/napari-allencell-segmenter) but fore the workflow we are proving out here we will want to call the `aicssegmentation` [package](https://github.com/AllenCell/aics-segmentation) directly.

#### ​The Allen Cell & Structure Segmenter 
​The Allen Cell & Structure Segmenter is a Python-based open source toolkit developed at the Allen Institute for Cell Science for 3D segmentation of intracellular structures in fluorescence microscope images. This toolkit brings together classic image segmentation and iterative deep learning workflows first to generate initial high-quality 3D intracellular structure segmentations and then to easily curate these results to generate the ground truths for building robust and accurate deep learning models. The toolkit takes advantage of the high replicate 3D live cell image data collected at the Allen Institute for Cell Science of over 30 endogenous fluorescently tagged human induced pluripotent stem cell (hiPSC) lines. Each cell line represents a different intracellular structure with one or more distinct localization patterns within undifferentiated hiPS cells and hiPSC-derived cardiomyocytes.

More details about Segmenter can be found at https://allencell.org/segmenter
In order to leverage the A
# IMPORTS

import  all nescessary packages

we are using `napari` for visualization, and `scipy` `ndimage` and `skimage` for analyzing the image files.  The underlying data format are `numpy` `ndarrays` and tools from  Allen Institute for Cell Science.


In [1]:

# this needs to be organzied to explain the imports
from pathlib import Path
import os

import numpy as np

import scipy
from scipy import ndimage as ndi

import napari

# function for core algorithm
import aicssegmentation

from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning
from aicssegmentation.core.MO_threshold import MO

from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice

from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters
from skimage import morphology

from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
# from aicsimageio import AICSImage

from napari.utils.notebook_display import nbscreenshot

#import .infer_subc.base
from infer_subc.base import *



# Get and load Image for processing
For testing purposes... TODO: build a nice wrapper for this.



Read the data into memeory from the `.czi` files.  (Note: there is also the 2D slice .tif file read for later comparision).  WE will also collect metatdata.


In [5]:
# assumptions:   .czi "unmixed" collections.
#czi_img_folder = f"{os.getenv('HOME')}/Projects/Imaging/mcz_subcell/data"

data_path = Path( f"{os.getenv('HOME')}/Projects/Imaging/mcz_subcell/data")
czi_img_folder = data_path/"raw"

list_img_files = lambda img_folder,f_type: [os.path.join(img_folder,f_name) for f_name in os.listdir(img_folder) if f_name.endswith(f_type)]

img_file_list = list_img_files(czi_img_folder,'.czi')
print(img_file_list[6])
#Read the data into memeory from the `.czi` files.  (Note: there is also the 2D slice .tif file read for later comparision).  WE will also collect metatdata.


/Users/ahenrie/Projects/Imaging/mcz_subcell/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR3_N11_Unmixed.czi


In [6]:

test_img_name = img_file_list[6]

img_data, meta_dict = read_input_image(test_img_name)

raw_meta_data, ome_types = get_raw_meta_data(meta_dict)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']



/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


Visualize the raw data file with [napari](https://napari.org)

In [7]:

viewer = napari.view_image(
    img_data,
    channel_axis=0,
    name=channel_names,
    scale=scale
)
viewer.scale_bar.visible = True


In [ ]:
##  need to save: 

# output_path, list_of_files